In [6]:
from pyspark.sql import SparkSession

In [8]:
spark = SparkSession.builder.appName('Aula').getOrCreate() 

In [9]:
spark

In [10]:
df_aluno = spark.createDataFrame([{"id":1,"nome":"fabio"},{"id":2,"nome":"andre"},{"id":3,"nome":"wagner"}])

In [11]:
df_aluno.show()

+---+------+
| id|  nome|
+---+------+
|  1| fabio|
|  2| andre|
|  3|wagner|
+---+------+



In [12]:
df_aluno.count()

3

In [70]:
df_user = spark.read.json('/datalake/raw/user')

In [71]:
df_user.count()

220

In [37]:
df_user.describe().show()

+-------+--------------------+----------+----------+------------------+---------+--------------+
|summary|               email|first_name|    gender|                id|last_name|      username|
+-------+--------------------+----------+----------+------------------+---------+--------------+
|  count|                  60|        60|        60|                60|       60|            60|
|   mean|                null|      null|      null|            4298.4|     null|          null|
| stddev|                null|      null|      null|2725.6791298030785|     null|          null|
|    min|alvin.hartmann@em...|     Alvin|   Agender|               234|     Batz|alvin.hartmann|
|    max|zane.watsica@emai...|      Zane|Polygender|              9762|  Ziemann|  zane.watsica|
+-------+--------------------+----------+----------+------------------+---------+--------------+



In [38]:
df_user.filter(df_user.id == 601).show(truncate=False)

+---------------------------+----------+--------+---+---------+-----------------+
|email                      |first_name|gender  |id |last_name|username         |
+---------------------------+----------+--------+---+---------+-----------------+
|sherwood.o'reilly@email.com|Sherwood  |Bigender|601|O'Reilly |sherwood.o'reilly|
+---------------------------+----------+--------+---+---------+-----------------+



In [39]:
df_user.where(df_user.username.endswith('e')).show()

+--------------------+----------+-----------+----+----------+---------------+
|               email|first_name|     gender|  id| last_name|       username|
+--------------------+----------+-----------+----+----------+---------------+
|david.ritchie@ema...|     David|   Bigender|9548|   Ritchie|  david.ritchie|
|etta.morissette@e...|      Etta|     Female|7229|Morissette|etta.morissette|
|emilia.brekke@ema...|    Emilia|       Male|1485|    Brekke|  emilia.brekke|
|philip.price@emai...|    Philip| Polygender|3771|     Price|   philip.price|
|carl.stracke@emai...|      Carl|       Male|2346|   Stracke|   carl.stracke|
|sean.boyle@email.com|      Sean|Genderqueer|1088|     Boyle|     sean.boyle|
+--------------------+----------+-----------+----+----------+---------------+



In [24]:
df_user.printSchema()

root
 |-- email: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- id: long (nullable = true)
 |-- last_name: string (nullable = true)
 |-- username: string (nullable = true)



In [40]:
df_end_e = df_user.where(df_user.username.endswith('e'))

In [43]:
df_end_e.count()

6

In [44]:
df_user.count()

60

In [48]:
df_user.write.csv('/datalake/process/csv',mode='overwrite',header=True, sep='@')

In [51]:
df_csv = spark.read.csv(path='/datalake/process/csv',header=True,sep='@')

In [52]:
df_csv.show()

+--------------------+-----------+-----------+----+-------------+-------------------+
|               email| first_name|     gender|  id|    last_name|           username|
+--------------------+-----------+-----------+----+-------------+-------------------+
|alvin.hartmann@em...|      Alvin| Polygender|5589|     Hartmann|     alvin.hartmann|
|colin.christianse...|      Colin| Polygender|2803| Christiansen| colin.christiansen|
|carlie.quitzon@em...|     Carlie|   Bigender|6175|      Quitzon|     carlie.quitzon|
|olin.runolfsdotti...|       Olin| Polygender|7087|Runolfsdottir| olin.runolfsdottir|
|svetlana.okuneva@...|   Svetlana|   Bigender|2999|      Okuneva|   svetlana.okuneva|
|sunni.hodkiewicz@...|      Sunni| Non-binary|9762|   Hodkiewicz|   sunni.hodkiewicz|
|carolin.schroeder...|    Carolin|Genderfluid|4011|    Schroeder|  carolin.schroeder|
|rolland.wyman@ema...|    Rolland|Genderfluid|1054|        Wyman|      rolland.wyman|
|jermaine.robel@em...|   Jermaine|Genderfluid|4783|   

In [53]:
df_user.write.parquet('/datalake/process/parquet')

In [55]:
df_parquet = spark.read.parquet('/datalake/process/parquet')

In [56]:
df_parquet.show()

+--------------------+----------+-----------+----+-------------+------------------+
|               email|first_name|     gender|  id|    last_name|          username|
+--------------------+----------+-----------+----+-------------+------------------+
|alvin.hartmann@em...|     Alvin| Polygender|5589|     Hartmann|    alvin.hartmann|
|colin.christianse...|     Colin| Polygender|2803| Christiansen|colin.christiansen|
|carlie.quitzon@em...|    Carlie|   Bigender|6175|      Quitzon|    carlie.quitzon|
|olin.runolfsdotti...|      Olin| Polygender|7087|Runolfsdottir|olin.runolfsdottir|
|svetlana.okuneva@...|  Svetlana|   Bigender|2999|      Okuneva|  svetlana.okuneva|
|sunni.hodkiewicz@...|     Sunni| Non-binary|9762|   Hodkiewicz|  sunni.hodkiewicz|
|carolin.schroeder...|   Carolin|Genderfluid|4011|    Schroeder| carolin.schroeder|
|rolland.wyman@ema...|   Rolland|Genderfluid|1054|        Wyman|     rolland.wyman|
|jermaine.robel@em...|  Jermaine|Genderfluid|4783|        Robel|    jermaine

In [57]:
df_parquet.join(df_csv,df_parquet.id == df_csv.id).show()

+--------------------+----------+-----------+----+-------------+------------------+--------------------+----------+-----------+----+-------------+------------------+
|               email|first_name|     gender|  id|    last_name|          username|               email|first_name|     gender|  id|    last_name|          username|
+--------------------+----------+-----------+----+-------------+------------------+--------------------+----------+-----------+----+-------------+------------------+
|alvin.hartmann@em...|     Alvin| Polygender|5589|     Hartmann|    alvin.hartmann|alvin.hartmann@em...|     Alvin| Polygender|5589|     Hartmann|    alvin.hartmann|
|colin.christianse...|     Colin| Polygender|2803| Christiansen|colin.christiansen|colin.christianse...|     Colin| Polygender|2803| Christiansen|colin.christiansen|
|carlie.quitzon@em...|    Carlie|   Bigender|6175|      Quitzon|    carlie.quitzon|carlie.quitzon@em...|    Carlie|   Bigender|6175|      Quitzon|    carlie.quitzon|
|oli

In [58]:
df_user.createOrReplaceTempView('usuario')

In [61]:
spark.sql('select * from usuario where username like "%e"').show()

+--------------------+----------+-----------+----+----------+---------------+
|               email|first_name|     gender|  id| last_name|       username|
+--------------------+----------+-----------+----+----------+---------------+
|david.ritchie@ema...|     David|   Bigender|9548|   Ritchie|  david.ritchie|
|etta.morissette@e...|      Etta|     Female|7229|Morissette|etta.morissette|
|emilia.brekke@ema...|    Emilia|       Male|1485|    Brekke|  emilia.brekke|
|philip.price@emai...|    Philip| Polygender|3771|     Price|   philip.price|
|carl.stracke@emai...|      Carl|       Male|2346|   Stracke|   carl.stracke|
|sean.boyle@email.com|      Sean|Genderqueer|1088|     Boyle|     sean.boyle|
+--------------------+----------+-----------+----+----------+---------------+



In [63]:
spark.sql('select * from json.`/datalake/raw/user` where id = 970').show()

+--------------------+----------+------+---+----------+------------------+
|               email|first_name|gender| id| last_name|          username|
+--------------------+----------+------+---+----------+------------------+
|nicolas.macejkovi...|   Nicolas|Female|970|Macejkovic|nicolas.macejkovic|
|kelley.franecki@e...|    Kelley|  Male|970|  Franecki|   kelley.franecki|
+--------------------+----------+------+---+----------+------------------+



In [64]:
url = 'jdbc:postgresql://postgres:5432/dvdrental'
prop = {"user":"admin",
       "password":"admin",
       "driver":"org.postgresql.Driver"}

In [65]:
df_city = spark.read.jdbc(url=url,properties=prop,table='public.city')

In [67]:
df_city.count()

653

In [72]:
df_user.write \
    .jdbc(url=url,properties=prop,table='public.userapi')

In [73]:
p = df_user.toPandas()

In [74]:
p

,email,first_name,gender,id,last_name,username
0,detra.pfannerstill@email.com,Detra,Female,4768,Pfannerstill,detra.pfannerstill
1,rigoberto.homenick@email.com,Rigoberto,Genderfluid,4217,Homenick,rigoberto.homenick
2,milton.powlowski@email.com,Milton,Genderfluid,3653,Powlowski,milton.powlowski
3,carma.cruickshank@email.com,Carma,Polygender,5787,Cruickshank,carma.cruickshank
4,brendon.breitenberg@email.com,Brendon,Female,7703,Breitenberg,brendon.breitenberg
...,...,...,...,...,...,...
215,jonas.murray@email.com,Jonas,Agender,1537,Murray,jonas.murray
216,mervin.auer@email.com,Mervin,Bigender,8391,Auer,mervin.auer
217,sal.strosin@email.com,Sal,Female,2451,Strosin,sal.strosin
218,hye.schmidt@email.com,Hye,Bigender,3524,Schmidt,hye.schmidt


In [75]:
k = df_user.to_koalas()

,email,first_name,gender,id,last_name,username
0,detra.pfannerstill@email.com,Detra,Female,4768,Pfannerstill,detra.pfannerstill
1,rigoberto.homenick@email.com,Rigoberto,Genderfluid,4217,Homenick,rigoberto.homenick
2,milton.powlowski@email.com,Milton,Genderfluid,3653,Powlowski,milton.powlowski
3,carma.cruickshank@email.com,Carma,Polygender,5787,Cruickshank,carma.cruickshank
4,brendon.breitenberg@email.com,Brendon,Female,7703,Breitenberg,brendon.breitenberg
5,hugh.mccullough@email.com,Hugh,Non-binary,1311,McCullough,hugh.mccullough
6,claudio.carroll@email.com,Claudio,Non-binary,3049,Carroll,claudio.carroll
7,leandro.romaguera@email.com,Leandro,Genderfluid,9522,Romaguera,leandro.romaguera
8,clorinda.lindgren@email.com,Clorinda,Bigender,8511,Lindgren,clorinda.lindgren
9,cornelia.quitzon@email.com,Cornelia,Genderfluid,778,Quitzon,cornelia.quitzon
